In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb as ddb
import os

In [11]:
raw_path="/Users/dhruvyellanki/Documents/Projects/Demand Forecasting-and-CTR-Prediction-System_Project_file/code_file/data/raw"
processed_path="/Users/dhruvyellanki/Documents/Projects/Demand Forecasting-and-CTR-Prediction-System_Project_file/code_file/data/processed"
interim_path="/Users/dhruvyellanki/Documents/Projects/Demand Forecasting-and-CTR-Prediction-System_Project_file/code_file/data/final"

## EXPLORING THE DATA

In [12]:
# Load each CSV
orders_raw = pd.read_csv(os.path.join(raw_path, "orders.csv"))
order_products_train_raw = pd.read_csv(os.path.join(raw_path, "order_products__train.csv"))
order_products_prior_raw = pd.read_csv(os.path.join(raw_path, "order_products__prior.csv"))
products_raw = pd.read_csv(os.path.join(raw_path, "products.csv"))
aisles_raw = pd.read_csv(os.path.join(raw_path, "aisles.csv"))
departments_raw = pd.read_csv(os.path.join(raw_path, "departments.csv"))

In [13]:
# Converting data to parquet format
orders_raw.to_parquet(os.path.join(processed_path, "orders.parquet"))
order_products_train_raw.to_parquet(os.path.join(processed_path, "order_products_train.parquet"))
order_products_prior_raw.to_parquet(os.path.join(processed_path, "order_products_prior.parquet"))
products_raw.to_parquet(os.path.join(processed_path, "products.parquet"))
aisles_raw.to_parquet(os.path.join(processed_path, "aisles.parquet"))
departments_raw.to_parquet(os.path.join(processed_path, "departments.parquet"))

In [14]:
# Loading the processed data
orders = pd.read_parquet(os.path.join(processed_path, "orders.parquet"))
order_products_train = pd.read_parquet(os.path.join(processed_path, "order_products_train.parquet"))
order_products_prior = pd.read_parquet(os.path.join(processed_path, "order_products_prior.parquet"))
products = pd.read_parquet(os.path.join(processed_path, "products.parquet"))
aisles = pd.read_parquet(os.path.join(processed_path, "aisles.parquet"))
departments = pd.read_parquet(os.path.join(processed_path, "departments.parquet"))

### orders dataset
These are the order records – who ordered, when, and how often (Tells us when and how often people shop).

In [15]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [19]:
orders.shape

(3421083, 7)

In [24]:
orders.duplicated(subset=['order_id']).sum()

0

In [25]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


### order_products__train dataset
These are the shopping basket details – what people actually bought in each order -last order used for training (Shows which items are popular or frequently re-bought).

### order_products__prior dataset
This file contains all the historical order details from users before their latest labeled training order -all the orders before that — kind of like their full shopping history (matters because it contains each user's full purchase history, which is essential for learning reorder patterns and building accurate forecasting or CTR features.)

### products dataset
This is the master list of all the products sold, like “Organic Bananas” or “Chocolate Ice Cream” (Connects items to their category and where they are in the store).

### aisles dataset
Think of this as the small sections inside a store aisle like "yogurt", "toothpaste", or "chips" ( Tells us what kind of product it is (subcategory)).

### departments dataset
These are the big categories in a store like "dairy", "beverages", or "household" (Helps us group products into broad types)